In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings 
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *
from statsforecast import StatsForecast
from statsforecast.models import (
    Naive,WindowAverage, ARIMA, 
    AutoARIMA,SeasonalNaive,HoltWinters,
    CrostonClassic as Croston, HistoricAverage,DynamicOptimizedTheta as DOT,
    SeasonalNaive
)
from dataclasses import dataclass
from typing import Optional, List
from itertools import product

warnings.filterwarnings("ignore")  # To ignore warnings from pandas/numpy

In [ ]:
user="Lilian"

In [ ]:
#New data class created to handle configuration parameters
@dataclass
class ForecastConfig:
    
    # Forecast parameters
    h: int = 8                          
    season_length: int = 4              
    
    # Cross-validation parameters
    n_windows: int = 2                  
    step_size: Optional[int] = None     
    
    # Train-test split parameters
    train_size: Optional[int] = None    # Use all available data except test
    test_size: Optional[int] = None     # Auto-set to h in __post_init__
    
    # Plotting parameters
    n_samples: int = 4                  # Plot 4 random samples
    models_to_plot: Optional[List[str]] = None  # 
    
    # Other settings
    confidence_level: int = 95          # 95% confidence intervals
    n_jobs: int = -1                    
    
    def __post_init__(self):
        
        if self.step_size is None:
            self.step_size = self.h
        if self.test_size is None:
            self.test_size = self.h
        if self.models_to_plot is None:
            self.models_to_plot = ['Naive', 'ARIMA_manual', 'SARIMA']

In [ ]:

def get_models(config):
    
    models = [Naive(), HistoricAverage(), WindowAverage(window_size=4),
        SeasonalNaive(season_length=4), ARIMA(order=(1, 1, 1), alias="ARIMA_manual"),
        AutoARIMA(seasonal=True, season_length=4, alias="SARIMA"),
    ]
    return models

def train_and_forecast(df,target_name, config):
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    
    forecasts_df = sf.forecast(df=df, h=config.h, level=[config.confidence_level])
    return forecasts_df, sf

def evaluate_train_test(df, target_name, config):

    if config.train_size is None:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[:-config.test_size]
        ).reset_index(drop=True)
    else:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[-(config.train_size + config.test_size):-config.test_size]
        ).reset_index(drop=True)
    
    test = df.groupby('unique_id').apply(
        lambda x: x.iloc[-config.test_size:]
    ).reset_index(drop=True)
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    sf.fit(df=train)
    preds = sf.predict(h=config.h)

    preds_df = pd.merge(test, preds.reset_index(), on=['ds', 'unique_id'], how='left')
    models = [col for col in preds.columns if col not in ['unique_id', 'ds']]
    
    eval_df = evaluate(preds_df, metrics=[mae, mse, rmse], models=models)
    
    mae_df = eval_df[eval_df['metric'] == 'mae'].copy()
    mae_df['best_model'] = mae_df[models].idxmin(axis=1)
    
    print(f"\n📈 Best Models (Train-Test Split - based on MAE):")
    print(mae_df['best_model'].value_counts())
    
    return eval_df, preds_df, train, test  

def evaluate_model_cross(df, target_name, config):
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )

    print(f"   Running cross-validation...")
    cv_df = sf.cross_validation(
        df=df,
        h=config.h,
        n_windows=config.n_windows,
        step_size=config.step_size
    )
    
    # Define model columns
    exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric']
    model_cols = [col for col in cv_df.columns if col not in exclude_cols]

    # Evaluate per cutoff window
    all_results = []
    
    for cutoff in cv_df['cutoff'].unique():
        cutoff_data = cv_df[cv_df['cutoff'] == cutoff]
        
        # Evaluate all metrics for this cutoff
        cutoff_eval = evaluate(cutoff_data, metrics=[mae, mse, rmse], models=model_cols)
        cutoff_eval['cutoff'] = cutoff
        
        # Add best_model column (lowest value per row, regardless of metric)
        cutoff_eval['best_model'] = cutoff_eval[model_cols].idxmin(axis=1)
        cutoff_eval['best_value'] = cutoff_eval[model_cols].min(axis=1)
        
        all_results.append(cutoff_eval)
        
        # Print MAE summary for this cutoff
        cutoff_mae = cutoff_eval[cutoff_eval['metric'] == 'mae']
        print(f"\n   Cutoff {cutoff.strftime('%Y-%m-%d')} (MAE best models):")
        print(f"   {cutoff_mae['best_model'].value_counts().to_dict()}")

    # Combine all cutoff results into single dataframe
    eval_df = pd.concat(all_results, ignore_index=True)
    
    # Print overall summary
    mae_overall = eval_df[eval_df['metric'] == 'mae']
    print(f"\n📈 Overall Best Models (all cutoffs - based on MAE):")
    print(mae_overall['best_model'].value_counts())
    
    return eval_df, cv_df

def get_best_model_forecast(forecasts_df,evaluation_df_summary):

    if 'unique_id' not in forecasts_df.columns:
        forecasts_df = forecasts_df.reset_index()
    
    # Get best model info
    best_info = evaluation_df_summary[['unique_id', 'best_model']].drop_duplicates()
    
    result_rows = []
    
    for uid in forecasts_df['unique_id'].unique():
        # Get forecasts for this unique_id
        uid_forecasts = forecasts_df[forecasts_df['unique_id'] == uid].copy()
        best_model_row = best_info[best_info['unique_id'] == uid]
        
        if len(best_model_row) == 0:
            print(f"Warning: No best model found for {uid}, skipping")
            continue
        
        best_model = best_model_row['best_model'].values[0]
        
        # For each forecast period
        for _, row in uid_forecasts.iterrows():
            result_row = {
                'unique_id': uid,
                'ds': row['ds'],
                'best_model': best_model,
                'best_forecast': row.get(best_model, np.nan)
            }
            
            for level in [95]:
                lo_col_source = f'{best_model}-lo-{level}'
                hi_col_source = f'{best_model}-hi-{level}'
                
                result_row[f'best_forecast-lo-{level}'] = row.get(lo_col_source, np.nan)
                result_row[f'best_forecast-hi-{level}'] = row.get(hi_col_source, np.nan)
            
            result_rows.append(result_row)
    
    result = pd.DataFrame(result_rows)
    
    return result

def add_best_model_column(eval_traintest_df):

    eval_with_best = eval_traintest_df.copy()
    model_cols=[col for col in eval_with_best.columns if col not in ['unique_id', 'metric']]
    eval_with_best['best_model'] = eval_with_best[model_cols].idxmin(axis=1)

    return eval_with_best

def save_results_to_excel(forecasts_df, best_forecasts_df, evaluation_cv_df, evaluation_traintest_df, target_name, save_path=None):
    
    if save_path is None or save_path == "":
        save_path = os.getcwd()
    
    os.makedirs(save_path, exist_ok=True)
    
    prefix = target_name.lower().replace(' ', '_')
    excel_filename=os.path.join(save_path, f'{prefix}_forecast_results.xlsx')

    eval_traintest_with_best = add_best_model_column(evaluation_traintest_df)
    
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:

        eval_traintest_with_best.to_excel(writer, sheet_name='Evaluation_TrainTest', index=False)
        forecasts_df.reset_index().to_excel(writer, sheet_name='Forecast_models', index=False)
        best_forecasts_df.to_excel(writer, sheet_name='Best_model_forecasts', index=False)
        evaluation_cv_df.to_excel(writer, sheet_name='Cross-Validation', index=False)

    print(f"✓ Results saved to: {excel_filename}")
    
def plot_sample_forecasts(df, forecasts_df, target_name, config):
 
    unique_ids = df['unique_id'].unique()
    sample_ids = np.random.choice(unique_ids, min(config.n_samples, len(unique_ids)), replace=False)
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    forecast_cols = forecasts_df.columns.tolist()
    available_models = [col for col in forecast_cols if col not in ['unique_id', 'ds'] and not col.endswith(('-lo-80', '-hi-80', '-lo-95', '-hi-95'))]
    
    # If models_to_plot is specified, filter to only those models
    if config.models_to_plot is not None:  
        available_models = [m for m in available_models if m in config.models_to_plot]
    
    palette = plt.cm.get_cmap('viridis', len(available_models))
    model_styles = {
        model: {'color': palette(idx), 'marker': '.', 'linestyle': '-'}
        for idx, model in enumerate(available_models)
    }
    
    for idx, uid in enumerate(sample_ids):
        ax = axes[idx]
        
        # Historical data
        hist_data = df[df['unique_id'] == uid]
        ax.plot(hist_data['ds'], hist_data['y'], 'o-', 
                label='Historical', linewidth=1.5, color='grey', markersize=3)
        
        # Forecast data
        forecast_data = forecasts_df.reset_index()
        forecast_data = forecast_data[forecast_data['unique_id'] == uid]
        
        # Plot each available model
        for model in available_models:
            if model in forecast_data.columns:
                style = model_styles.get(model, {'color': 'gray', 'marker': 'o', 'linestyle': '-'})
                ax.plot(forecast_data['ds'], forecast_data[model], 
                       marker=style['marker'], 
                       linestyle=style['linestyle'],
                       color=style['color'],
                       label=model, 
                       linewidth=1.5,
                       markersize=3)
        
        ax.set_title(f'{uid}', fontsize=10, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel(target_name)
        ax.legend(loc='best', fontsize=8)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    filename = f'{target_name.lower().replace(" ", "_")}_sample_forecasts.png'
    plt.show()
    plt.close()

def plot_train_test_forecasts(df, train_df, test_df, preds_df, target_name, config, ids_to_plot=None):

    
    if ids_to_plot is None:
        unique_ids = df['unique_id'].unique()
        ids_to_plot = np.random.choice(
            unique_ids, 
            min(config.n_samples, len(unique_ids)),  # ← Use config.n_samples
            replace=False
        )
    
    n_plots = len(ids_to_plot); n_cols = 2; n_rows = (n_plots + 1) // 2
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5 * n_rows))
    if n_plots == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    # Get available models from preds_df
    all_model_cols = [col for col in preds_df.columns 
                      if col not in ['unique_id', 'ds', 'y', 'cutoff']]
    
    # Print available models for debugging
    print(f"\n🔍 Available models in predictions: {all_model_cols}")
    
    # Filter to selected models if specified
    if config.models_to_plot is not None:  # ← Use config.models_to_plot
        # Try exact match first
        model_cols = [m for m in all_model_cols if m in config.models_to_plot]
        
        # If no exact matches, try case-insensitive partial matching
        if len(model_cols) == 0:
            model_cols = []
            for requested in config.models_to_plot:  # ← Use config.models_to_plot
                for available in all_model_cols:
                    if requested.lower() in available.lower() or available.lower() in requested.lower():
                        model_cols.append(available)
            model_cols = list(set(model_cols))  # Remove duplicates
        
        if len(model_cols) == 0:
            print(f"⚠️  Warning: None of the requested models {config.models_to_plot} found.")
            print(f"   Available models: {all_model_cols}")
            print(f"   Plotting all available models instead.")
            model_cols = all_model_cols
        else:
            print(f"✓ Plotting models: {model_cols}")
    else:
        model_cols = all_model_cols
        print(f"✓ Plotting all {len(model_cols)} models")
    
    for idx, uid in enumerate(ids_to_plot):
        ax = axes[idx]
        
        # Get data for this unique_id
        hist_data = df[df['unique_id'] == uid].sort_values('ds')
        train_data = train_df[train_df['unique_id'] == uid].sort_values('ds')
        test_data = test_df[test_df['unique_id'] == uid].sort_values('ds')
        pred_data = preds_df[preds_df['unique_id'] == uid].sort_values('ds')
        
        # Plot training data (historical)
        ax.plot(train_data['ds'], train_data['y'], 
                'o-', color='Grey', linewidth=2, markersize=5, 
                label='Training Data', zorder=3)
        
        # Plot ACTUAL test data
        ax.plot(test_data['ds'], test_data['y'], 
                'o-', color='darkgreen', linewidth=2.5, markersize=7, 
                label='Actual Test Values', zorder=5)

            # Plot predictions for each selected model
        palette = plt.cm.get_cmap('viridis', len(model_cols))
        for model_idx, model in enumerate(model_cols):
            if model in pred_data.columns:
                ax.plot(pred_data['ds'], pred_data[model], 
                       's--', color=palette(model_idx), linewidth=2, 
                       markersize=6, label=f'{model}', 
                       alpha=0.8, zorder=4)
        
        # Add vertical line to separate train/test
        if len(train_data) > 0:
            train_end = train_data['ds'].max()
            ax.axvline(x=train_end, color='red', linestyle=':', linewidth=2.5, 
                      label='Train/Test Split', alpha=0.7, zorder=2)
        
        # Add shaded region for test period
        if len(test_data) > 0:
            test_start = test_data['ds'].min()
            test_end = test_data['ds'].max()
            ax.axvspan(test_start, test_end, alpha=0.1, color='yellow', 
                      label='Test Period', zorder=1)
        
        ax.set_title(f'{uid}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Date', fontsize=10)
        ax.set_ylabel(target_name, fontsize=10)
        ax.legend(loc='best', fontsize=8, framealpha=0.9)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
    
    # Hide extra subplots if n_plots is odd
    for idx in range(n_plots, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    filename = f'{target_name.lower().replace(" ", "_")}_train_test_forecasts.png'
    plt.show()
    plt.close()

In [12]:
def sensitivity_analysis(df, target_name, param_grid=None, save_path=None):
    
    # Default parameter grid if not provided
    if param_grid is None:
        param_grid = {
            'h': [4, 8],
            'test_size': [4, 8],
            'train_size': [None, 20, 28],  # None = use all available
            'n_windows': [2, 3, 4],
            'step_size': [None, 2, 4],  # None = defaults to h
        }
    
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))
    
    print(f"   Total configurations to test: {len(combinations)}")
    print(f"   Parameters: {param_names}")
    
    all_results = []
    detailed_results = []
    
    for i, combo in enumerate(combinations):
        params = dict(zip(param_names, combo))
        
        if params.get('test_size') and params.get('h'):
            if params['test_size'] < params['h']:
                continue
        
        # Check if we have enough data for this configuration
        min_data_needed = (params.get('train_size') or 0) + (params.get('test_size') or params['h'])
        min_series_length = df.groupby('unique_id').size().min()
        
        if params.get('train_size') and params.get('n_windows'):
            cv_data_needed = params['train_size'] + params['h'] * params['n_windows']
            if cv_data_needed > min_series_length:
                print(f"   ⚠️  Skipping config {i+1}: insufficient data (need {cv_data_needed}, have {min_series_length})")
                continue
        
        print(f"\n📊 Configuration {i+1}/{len(combinations)}: {params}")
        try:
            # Create config object
            config = ForecastConfig(
                h=params.get('h', 8),
                train_size=params.get('train_size'),
                test_size=params.get('test_size'),
                n_windows=params.get('n_windows', 2),
                step_size=params.get('step_size'),
                season_length=4,
                n_samples=4,
                confidence_level=95,
                n_jobs=-1
            )
            
            # Run train-test evaluation
            eval_traintest, preds_traintest, train_df, test_df = evaluate_train_test(
                df, target_name, config
            )
            
            # Run cross-validation
            eval_cv, cv_df = evaluate_model_cross(df, target_name, config)
            
            # Extract MAE results
            mae_cv = eval_cv[eval_cv['metric'] == 'mae'].copy()
            mae_traintest = eval_traintest[eval_traintest['metric'] == 'mae'].copy()
            
            # Get model columns
            exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'best_model', 'best_value']
            model_cols = [col for col in mae_cv.columns if col not in exclude_cols]
            
            # Add best_model to train-test if not present
            if 'best_model' not in mae_traintest.columns:
                mae_traintest['best_model'] = mae_traintest[model_cols].idxmin(axis=1)
            
            # Summarize results for this configuration
            for uid in df['unique_id'].unique():
                # CV results
                uid_cv = mae_cv[mae_cv['unique_id'] == uid]
                if len(uid_cv) > 0:
                    # Get most frequent best model across cutoffs
                    cv_best_model = uid_cv['best_model'].mode().iloc[0] if len(uid_cv) > 0 else None
                    cv_best_count = (uid_cv['best_model'] == cv_best_model).sum()
                    cv_total = len(uid_cv)
                    cv_consistency = cv_best_count / cv_total if cv_total > 0 else 0
                else:
                    cv_best_model = None
                    cv_consistency = 0
                
                # Train-test results
                uid_traintest = mae_traintest[mae_traintest['unique_id'] == uid]
                traintest_best_model = uid_traintest['best_model'].iloc[0] if len(uid_traintest) > 0 else None
                
                result_row = {
                    'unique_id': uid,
                    'config_id': i + 1,
                    **params,
                    'cv_best_model': cv_best_model,
                    'cv_consistency': cv_consistency,  # How often this model won across cutoffs
                    'traintest_best_model': traintest_best_model,
                    'cv_traintest_agree': cv_best_model == traintest_best_model,
                }
                
                # Add MAE values for each model (from CV)
                for model in model_cols:
                    model_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    result_row[f'{model}_mae'] = model_mae
                
                all_results.append(result_row)
            
            # Store detailed results
            detailed_results.append({
                'config_id': i + 1,
                'params': params,
                'eval_cv': eval_cv,
                'eval_traintest': eval_traintest,
                'cv_df': cv_df
            })
            
            print(f"   ✓ Completed successfully")
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            continue
    
    # Create summary dataframe
    results_df = pd.DataFrame(all_results)
    
    # Print summary
    print("\n" + "=" * 70); print("📈 SENSITIVITY ANALYSIS SUMMARY"); print("=" * 70)
    
    if len(results_df) > 0:
        # Overall best model frequency
        print("\n🏆 Best Model Frequency (Cross-Validation):")
        print(results_df['cv_best_model'].value_counts())
        
        print("\n🏆 Best Model Frequency (Train-Test):")
        print(results_df['traintest_best_model'].value_counts())
        
        # Agreement between CV and train-test
        agreement_rate = results_df['cv_traintest_agree'].mean() * 100
        print(f"\n🤝 CV vs Train-Test Agreement: {agreement_rate:.1f}%")
        
        # Consistency per unique_id
        print("\n📊 Model Selection Consistency by Series:")
        consistency_summary = results_df.groupby('unique_id').agg({
            'cv_best_model': lambda x: x.mode().iloc[0] if len(x) > 0 else None,
            'cv_consistency': 'mean',
            'cv_traintest_agree': 'mean'
        }).round(3)
        print(consistency_summary)
    
    # Save results if path provided
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        filename = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_sensitivity_analysis.xlsx')
        
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            results_df.to_excel(writer, sheet_name='Summary', index=False)
            
            # Pivot table: best model by config and unique_id
            if len(results_df) > 0:
                pivot_cv = results_df.pivot_table(
                    index='unique_id', 
                    columns='config_id', 
                    values='cv_best_model', 
                    aggfunc='first'
                )
                pivot_cv.to_excel(writer, sheet_name='CV_BestModel_Pivot')
                
                pivot_traintest = results_df.pivot_table(
                    index='unique_id', 
                    columns='config_id', 
                    values='traintest_best_model', 
                    aggfunc='first'
                )
                pivot_traintest.to_excel(writer, sheet_name='TrainTest_BestModel_Pivot')
        
        print(f"\n✓ Results saved to: {filename}")
    
    return results_df, detailed_results

def analyze_models(results_df, target_name,save_path=None):
    
    unique_ids = results_df['unique_id'].unique()
    params_to_analyze = ['h', 'test_size', 'train_size', 'n_windows', 'step_size']
    params_available = [p for p in params_to_analyze if p in results_df.columns]
    
    mae_cols = [col for col in results_df.columns if col.endswith('_mae')]
    model_names = [col.replace('_mae', '') for col in mae_cols]
    
    #This is for both recommendations and parameter impacts 
    all_recommendations = []; all_parameter_impacts = []
    
    for uid in unique_ids:
        uid_data = results_df[results_df['unique_id'] == uid].copy()
        print(f"\n🔍 Analyzing Unique ID: {uid}")
        #  Most frequent CV best model
        cv_mode = uid_data['cv_best_model'].mode()
        cv_best = cv_mode.iloc[0] if len(cv_mode) > 0 else None
        cv_freq = (uid_data['cv_best_model'] == cv_best).sum() / len(uid_data)
        
        # Most frequent train-test best model
        tt_mode = uid_data['traintest_best_model'].mode()
        tt_best = tt_mode.iloc[0] if len(tt_mode) > 0 else None
        tt_freq = (uid_data['traintest_best_model'] == tt_best).sum() / len(uid_data)
        
        # Overall agreement rate
        agreement_rate = uid_data['cv_traintest_agree'].mean()
        
        # Average consistency across CV windows
        avg_consistency = uid_data['cv_consistency'].mean()
        
        # Determine recommendation and confidence
        if cv_best == tt_best and cv_freq >= 0.7 and avg_consistency >= 0.7:
            confidence = 'High'
            recommended_model = cv_best
            reason = "CV and Train-Test agree, high frequency and consistency"
        elif cv_best == tt_best and cv_freq >= 0.5:
            confidence = 'Medium-High'
            recommended_model = cv_best
            reason = "CV and Train-Test agree with moderate frequency"
        elif cv_freq >= 0.6:
            confidence = 'Medium'
            recommended_model = cv_best
            reason = f"CV favors {cv_best} ({cv_freq:.0%}), but Train-Test prefers {tt_best}"
        elif tt_freq >= 0.6:
            confidence = 'Medium'
            recommended_model = tt_best
            reason = f"Train-Test favors {tt_best} ({tt_freq:.0%}), but CV prefers {cv_best}"
        else:
            confidence = 'Low'
            # Use model with lowest average MAE
            if mae_cols:
                avg_maes = uid_data[mae_cols].mean()
                recommended_model = avg_maes.idxmin().replace('_mae', '')
            else:
                recommended_model = cv_best
            reason = "No clear winner - using lowest average MAE"
        
        # Get average MAE for recommended model
        rec_mae_col = f'{recommended_model}_mae'
        avg_mae = uid_data[rec_mae_col].mean() if rec_mae_col in uid_data.columns else None
        
        # Store recommendation
        recommendation = {
            'unique_id': uid,
            'recommended_model': recommended_model,
            'confidence': confidence,
            'reason': reason,
            'cv_best_model': cv_best,
            'cv_frequency': cv_freq,
            'cv_consistency': avg_consistency,
            'traintest_best_model': tt_best,
            'traintest_frequency': tt_freq,
            'cv_traintest_agreement': agreement_rate,
            'recommended_model_avg_mae': avg_mae
        }
        all_recommendations.append(recommendation)
        
        print(f"\nRECOMMENDATION: {recommended_model}"); print(f"   Confidence: {confidence}")
        print(f"   Reason: {reason}"); print(f"   CV Best: {cv_best} ({cv_freq:.0%} of configs)")
        print(f"   Train-Test Best: {tt_best} ({tt_freq:.0%} of configs)"); print(f"   CV-TT Agreement: {agreement_rate:.0%}")
        if avg_mae:
            print(f"   Avg MAE: {avg_mae:,.0f}")

        print(f"\nPARAMETER IMPACT:")
        
        for param in params_available:
            param_values = uid_data[param].dropna().unique()
            
            if len(param_values) <= 1:
                continue
            
            print(f"\n   {param}:")
            
            for val in sorted(param_values):
                val_data = uid_data[uid_data[param] == val]
                if len(val_data) == 0:
                    continue
                
                # Best model for this parameter value
                val_cv_best = val_data['cv_best_model'].mode()
                val_cv_best = val_cv_best.iloc[0] if len(val_cv_best) > 0 else None
                val_cv_freq = (val_data['cv_best_model'] == val_cv_best).sum() / len(val_data)
                
                # Average MAE for best model at this parameter value
                if val_cv_best and f'{val_cv_best}_mae' in val_data.columns:
                    val_avg_mae = val_data[f'{val_cv_best}_mae'].mean()
                    mae_str = f", Avg MAE: {val_avg_mae:,.0f}"
                else:
                    val_avg_mae = None
                    mae_str = ""
                
                print(f"      {param}={val}: {val_cv_best} ({val_cv_freq:.0%}){mae_str}")
                
                # Store parameter impact
                all_parameter_impacts.append({
                    'unique_id': uid,
                    'parameter': param,
                    'value': val,
                    'best_model': val_cv_best,
                    'frequency': val_cv_freq,
                    'avg_mae': val_avg_mae,
                    'n_configs': len(val_data)
                })
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        
        # Plot 1: CV Best Model Frequency
        ax1 = axes[0, 0]
        cv_counts = uid_data['cv_best_model'].value_counts()
        colors = ['green' if m == recommended_model else 'steelblue' for m in cv_counts.index]
        cv_counts.plot(kind='bar', ax=ax1, color=colors, edgecolor='black')
        ax1.set_title('CV Best Model Frequency', fontweight='bold')
        ax1.set_xlabel('Model')
        ax1.set_ylabel('Count')
        ax1.tick_params(axis='x', rotation=45)
        for i, (idx, val) in enumerate(cv_counts.items()):
            ax1.annotate(f'{val/len(uid_data):.0%}', xy=(i, val), ha='center', va='bottom')
        
        # Plot 2: Train-Test Best Model Frequency
        ax2 = axes[0, 1]
        tt_counts = uid_data['traintest_best_model'].value_counts()
        colors = ['green' if m == recommended_model else 'darkorange' for m in tt_counts.index]
        tt_counts.plot(kind='bar', ax=ax2, color=colors, edgecolor='black')
        ax2.set_title('Train-Test Best Model Frequency', fontweight='bold')
        ax2.set_xlabel('Model')
        ax2.set_ylabel('Count')
        ax2.tick_params(axis='x', rotation=45)
        for i, (idx, val) in enumerate(tt_counts.items()):
            ax2.annotate(f'{val/len(uid_data):.0%}', xy=(i, val), ha='center', va='bottom')
        
        # Plot 3: Average MAE by Model
        ax3 = axes[0, 2]
        if mae_cols:
            avg_maes = uid_data[mae_cols].mean().sort_values()
            colors = ['green' if m.replace('_mae', '') == recommended_model else 'gray' 
                     for m in avg_maes.index]
            avg_maes.plot(kind='barh', ax=ax3, color=colors, edgecolor='black')
            ax3.set_title('Average MAE by Model', fontweight='bold')
            ax3.set_xlabel('MAE')
            ax3.set_yticklabels([l.get_text().replace('_mae', '') for l in ax3.get_yticklabels()])
            for i, val in enumerate(avg_maes):
                ax3.annotate(f'{val:,.0f}', xy=(val, i), ha='left', va='center', fontsize=9)
        
        # Plot 4-6: Parameter Impact (up to 3 parameters)
        for idx, param in enumerate(params_available[:3]):
            ax = axes[1, idx]
            param_pivot = uid_data.groupby([param, 'cv_best_model']).size().unstack(fill_value=0)
            param_pivot.plot(kind='bar', ax=ax, edgecolor='black', width=0.8)
            ax.set_title(f'Best Model by {param}', fontweight='bold')
            ax.set_xlabel(param)
            ax.set_ylabel('Count')
            ax.legend(title='Model', fontsize=8)
            ax.tick_params(axis='x', rotation=0)
        
        # Hide unused subplots
        for idx in range(len(params_available[:3]), 3):
            axes[1, idx].set_visible(False)
        
        plt.suptitle(f'Model Analysis: {target_name} - {uid}\n'
                    f'Recommended: {recommended_model} (Confidence: {confidence})', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        if save_path:
            os.makedirs(save_path, exist_ok=True)
            filename = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_analysis_{uid}.png')
            plt.savefig(filename, dpi=150, bbox_inches='tight')
            print(f"\n   ✓ Saved: {filename}")
        
        plt.show()
        plt.close()
    recommendations_df = pd.DataFrame(all_recommendations)
    parameter_impact_df = pd.DataFrame(all_parameter_impacts)
    
    print("\nFINAL RECOMMENDATIONS SUMMARY")
    
    summary_cols = ['unique_id', 'recommended_model', 'confidence', 'cv_frequency', 'traintest_frequency', 'cv_traintest_agreement']
    print(recommendations_df[summary_cols].to_string(index=False))
    
    # Confidence distribution
    print(f"\nConfidence Distribution:")
    print(recommendations_df['confidence'].value_counts().to_string())
    
    # Model recommendation distribution
    print(f"\nRecommended Models Distribution:")
    print(recommendations_df['recommended_model'].value_counts().to_string())
    
    # Parameter sensitivity summary
    print(f"\nParameter Sensitivity Summary:")
    param_sensitivity = parameter_impact_df.groupby(['unique_id', 'parameter']).apply(
        lambda x: x['best_model'].nunique()
    ).unstack(fill_value=0)
    print("   (Number of different best models across parameter values)")
    print(param_sensitivity.to_string())
    
    # Save results to Excel
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        filename = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_recommendations.xlsx')
        
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            recommendations_df.to_excel(writer, sheet_name='Recommendations', index=False)
            parameter_impact_df.to_excel(writer, sheet_name='Parameter_Impact', index=False)
            param_sensitivity.to_excel(writer, sheet_name='Param_Sensitivity')
        
        print(f"\n✓ Results saved to: {filename}")
    
    return recommendations_df, parameter_impact_df

### Population Forecast

In [ ]:
def load_data(filepath, states=None):
    
    df=pd.read_csv(filepath)
    #Filtering for certain years
    df = df[(df['Period'] >= '2017Q1') & (df['Period'] <= '2024Q4')].copy()
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df = df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].unique()} state(s)")
        if len(df) == 0:
            raise ValueError("No data available after filtering by states.")    
    
    df['unique_id']=df['State']
    df['ds']=pd.to_datetime(df['Period'])
    df=df.sort_values(['unique_id','ds']).reset_index(drop=True)
    df_pop=df[['unique_id','ds','Population']].copy()
    df_pop.columns = ['unique_id', 'ds', 'y']
    

    return df_pop,df

In [ ]:
def main_pop(filepath, states, config, save_path=None, run_cv=True):

    print("\n1. Loading and preparing pop data")
    df_pop, df_original = load_data(filepath, states=states)

    print(f"\nTotal states: {df_pop['unique_id'].nunique()}")
    min_date = df_pop['ds'].min(); max_date = df_pop['ds'].max()
    print(f"Date range: {min_date.year}-Q{min_date.quarter} to {max_date.year}-Q{max_date.quarter}")
    print(f"States included: {states}")

    print("\n2. Training models and generating forecasts for Population")
    forecasts_pop, sf_pop = train_and_forecast(df_pop, "Population", config)

    print("\n3. Evaluating models on training/test split for Population")
    eval_traintest_pop, preds_traintest_pop, train_pop, test_pop = evaluate_train_test(
        df_pop, "Population", config
    )

    if run_cv:
        
        print("\n4. Running cross-validation for Population...")
        eval_cv_pop, cv_pop = evaluate_model_cross(
            df_pop, "Population", config)
        mae_summary_pop = eval_cv_pop[eval_cv_pop['metric'] == 'mae']
        best_forecasts_pop = get_best_model_forecast(forecasts_pop, mae_summary_pop)
        
        print("\n5. Selecting best model forecasts for Population...")
        best_forecasts_pop = get_best_model_forecast(forecasts_pop, mae_summary_pop)

        print("\n6. Saving results to Excel for Population...")
        save_results_to_excel(
            forecasts_pop, 
            best_forecasts_pop, 
            eval_cv_pop, 
            eval_traintest_pop, 
            "Population", 
            save_path=save_path
        )
    else:
        eval_cv_pop = None
        best_forecasts_pop = forecasts_pop.reset_index()
        print("Skipping cross-validation")
    
    # Plot forecasts
    plot_sample_forecasts(df_pop, forecasts_pop, "Population", config)
    
    # Plot train/test comparison
    print("\n📈 Generating train/test visualization...")
    plot_train_test_forecasts(
        df=df_pop,
        train_df=train_pop,
        test_df=test_pop,
        preds_df=preds_traintest_pop,
        target_name="Population",
        config=config,
        ids_to_plot=states
    )
    
    print("\n7. Compiling all results..."); print("✓ Done!")
    return {
        'population': {
            'forecasts': forecasts_pop,
            'best_forecasts': best_forecasts_pop if run_cv else forecasts_pop.reset_index(),
            'evaluation_cv': eval_cv_pop if run_cv else None,
            'evaluation_traintest': eval_traintest_pop,
            'predictions_traintest': preds_traintest_pop
        }
    }

In [ ]:
pop_csv_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\medi_pop.csv"
pop_save_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\Pop\\"

In [ ]:
#See first the class because some parameters are already fixed

config = ForecastConfig(
    h=8,                    
    season_length=4,        
    n_windows=2, 
    train_size=None,           
    test_size=8,            
    n_samples=4,            
    confidence_level=95,    
    models_to_plot=['Naive', 'ARIMA_manual', 'SARIMA']
)

results_pop = main_pop(
    filepath=pop_csv_path,
    states=['IN', 'MI', 'IL', 'OH'],
    config=config,              
    save_path=pop_save_path,
    run_cv=True
)

In [ ]:
df_pop, df_original = load_data(filepath=pop_csv_path, states=['IL', 'IN', 'MI', 'OH'])

param_grid = {
    'h': [4, 8],
    'test_size': [4, 8],
    'train_size': [20, 28, None],
    'n_windows': [2, 3],
    'step_size': [None, 4],
}

# Run sensitivity analysis
results_df, detailed_results = sensitivity_analysis(
    df=df_pop,
    target_name="Population",
    param_grid=param_grid,
    save_path=pop_save_path
)

# Run combined analysis and get recommendations
recommendations_df, parameter_impact_df = analyze_models(
    results_df=results_df,
    target_name="Population",
    save_path=pop_save_path
)

### Statistical Models - Dataset with pop

In [ ]:
def load_and_prepare_data(filepath, states=None):

    df = pd.read_csv(filepath)
    #Cutting the dataframe fo selected years
    df=df[(df['Period'] >= '2017Q1') & (df['Period'] <= '2024Q4')].copy()
    
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df=df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].nunique()} state(s).")

        if len(df) == 0:
            raise ValueError("No data available for the specified states.")
    
    df['unique_id'] = df['State'] + '_' + df['ATC2 Class']
    df['ds'] = pd.to_datetime(df['Period'])
    df = df.sort_values(['unique_id','ds']).reset_index(drop=True)

    #Preparing dataframes for statsforecast
    df_units=df[['unique_id','ds','Units Reimbursed']].copy()
    df_units.columns=['unique_id','ds','y']

    df_prescriptions=df[['unique_id','ds','Number of Prescriptions']].copy()
    df_prescriptions.columns=['unique_id','ds','y']

    return df_units, df_prescriptions, df

In [ ]:
def main(filepath, states=None, config=None, run_cv=True, save_path=None):
    
    # Create default config if not provided
    if config is None:
        config = ForecastConfig()
    
    print("\n1. Loading and preparing data...")
    df_units, df_prescriptions, df_original = load_and_prepare_data(filepath, states=states)
    
    # Display summary
    print(f"\nData Summary:")
    print(f"  • Total unique series: {df_units['unique_id'].nunique()}")
    min_date = df_units['ds'].min()
    max_date = df_units['ds'].max()
    print(f"  • Date range: {min_date.year}-Q{min_date.quarter} to {max_date.year}-Q{max_date.quarter}")
    if states is not None:
        print(f"  • Filtered states: {', '.join(states)}")
    

    print("\n" + "="*70)
    print("UNITS REIMBURSED")
    print("="*70)
    
    print("\n2. Training models and generating forecasts...")
    forecasts_units, sf_units = train_and_forecast(df_units, "Units Reimbursed", config)

    print("\n3. Evaluating models on training/test split...")
    eval_traintest_units, preds_traintest_units, train_units, test_units = evaluate_train_test(
        df_units, "Units Reimbursed", config
    )
    
    if run_cv:
        print("\n4. Running cross-validation...")
        eval_cv_units, cv_units, eval_summary_units, per_cutoff_units = evaluate_model_cross(
            df_units, "Units Reimbursed", config
        )
        
        print("\n5. Selecting best forecasts...")
        best_forecasts_units = get_best_model_forecast(forecasts_units, eval_summary_units)
        
        print("\n6. Saving results...")
        save_results_to_excel(
            forecasts_units, best_forecasts_units, 
            eval_cv_units, eval_traintest_units, 
            "Units Reimbursed", save_path=save_path
        )
    else:
        eval_cv_units = None
        per_cutoff_units = None
        best_forecasts_units = forecasts_units.reset_index()
        print("\n Skipping cross-validation (run_cv=False)")
    
    # Plot sample forecasts
    print("\nGenerating forecast plots...")
    plot_sample_forecasts(df_units, forecasts_units, "Units Reimbursed", config)
    plot_train_test_forecasts(df_units, train_units, test_units, preds_traintest_units, "Units Reimbursed", config)
    

    print("\n" + "="*70)
    print("NUMBER OF PRESCRIPTIONS")
    print("="*70)
    
    print("\n2. Training models and generating forecasts...")
    forecasts_prescriptions, sf_prescriptions = train_and_forecast(
        df_prescriptions, "Number of Prescriptions", config
    )

    print("\n3. Evaluating models on training/test split...")
    eval_traintest_prescriptions, preds_traintest_prescriptions, train_prescriptions, test_prescriptions = evaluate_train_test(
        df_prescriptions, "Number of Prescriptions", config
    )
    
    if run_cv:
        print("\n4. Running cross-validation...")
        eval_cv_prescriptions, cv_prescriptions, eval_summary_prescriptions, per_cutoff_prescriptions = evaluate_model_cross(
            df_prescriptions, "Number of Prescriptions", config
        )
        
        print("\n5. Selecting best forecasts...")
        best_forecasts_prescriptions = get_best_model_forecast(
            forecasts_prescriptions, eval_summary_prescriptions
        )
        
        print("\n6. Saving results...")
        save_results_to_excel(
            forecasts_prescriptions, best_forecasts_prescriptions, 
            eval_cv_prescriptions, eval_traintest_prescriptions, 
            "Number of Prescriptions", save_path=save_path
        )
    else:
        eval_cv_prescriptions = None
        per_cutoff_prescriptions = None
        best_forecasts_prescriptions = forecasts_prescriptions.reset_index()
        print("\n Skipping cross-validation (run_cv=False)")
    
    # Plot sample forecasts
    print("\nGenerating forecast plots...")
    plot_sample_forecasts(df_prescriptions, forecasts_prescriptions, "Number of Prescriptions", config)
    plot_train_test_forecasts(df_prescriptions, train_prescriptions, test_prescriptions, preds_traintest_prescriptions, "Number of Prescriptions", config)
    
    print("\n" + "="*70)
    print("✅ PIPELINE COMPLETE!")
    print("="*70)
    
    state_info = f"{', '.join(states)}" if states else "ALL"
    print(f"\nForecasts generated for state(s): {state_info}")
    print(f"Forecast horizon: {config.h} quarters ahead")
    
    if run_cv:
        print(f"\nAll files generated!")
    
    return {
        'units': {
            'forecasts': forecasts_units,
            'best_forecasts': best_forecasts_units if run_cv else None,
            'evaluation_cv': eval_cv_units if run_cv else None,
            'evaluation_traintest': eval_traintest_units,
            'predictions_traintest': preds_traintest_units,
            'per_cutoff_best': per_cutoff_units if run_cv else None,
        },
        'prescriptions': {
            'forecasts': forecasts_prescriptions,
            'best_forecasts': best_forecasts_prescriptions if run_cv else None,
            'evaluation_cv': eval_cv_prescriptions if run_cv else None,
            'evaluation_traintest': eval_traintest_prescriptions,
            'predictions_traintest': preds_traintest_prescriptions,
            'per_cutoff_best': per_cutoff_prescriptions if run_cv else None,
        }
    }

In [ ]:
path_save=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\TMF\\"
os.path.isdir(path_save)

In [ ]:
#path to the pre-built dataset
csv_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\P1_nopop.csv"
path_save=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\TMF\\"

if __name__ == "__main__":
    main(filepath=csv_path, states=['IN'], h=8, run_cv=True, n_windows=5,save_path=path_save)
  